In [7]:
import json
import os
import uuid
import numpy as np
import logging

from PIL import Image
from collections import defaultdict
from itertools import groupby

logger = logging.getLogger(__name__)

### Brush Export ###

class InputStream:
    def __init__(self, data):
        self.data = data
        self.i = 0

    def read(self, size):
        out = self.data[self.i:self.i + size]
        self.i += size
        return int(out, 2)


def access_bit(data, num):
    """ from bytes array to bits by num position
    """
    base = int(num // 8)
    shift = 7 - int(num % 8)
    return (data[base] & (1 << shift)) >> shift


def bytes2bit(data):
    """ get bit string from bytes data
    """
    return ''.join([str(access_bit(data, i)) for i in range(len(data) * 8)])


def decode_rle(rle, print_params: bool = False):
    """ from LS RLE to numpy uint8 3d image [width, height, channel]
    
    Args:
        print_params (bool, optional): If true, a RLE parameters print statement is suppressed
    """
    input = InputStream(bytes2bit(rle))
    num = input.read(32)
    word_size = input.read(5) + 1
    rle_sizes = [input.read(4) + 1 for _ in range(4)]
    
    if print_params:
        print('RLE params:', num, 'values', word_size, 'word_size', rle_sizes, 'rle_sizes')
        
    i = 0
    out = np.zeros(num, dtype=np.uint8)
    while i < num:
        x = input.read(1)
        j = i + 1 + input.read(rle_sizes[input.read(2)])
        if x:
            val = input.read(word_size)
            out[i:j] = val
            i = j
        else:
            while i < j:
                val = input.read(word_size)
                out[i] = val
                i += 1
    return out

# create gambar disini
def convert_to_png(json_path):
  """ Convert the .json from brushlabels
  
  Args: json_path = path to the json file from frontend module
  """
data = json.load(open('../static/json_annotation/img_back.json'))
for d in data:

    width = d['original_width']
    height = d['original_height']
    brushlabels = str(d['value']['brushlabels'])

    img_name = brushlabels.translate({ord(i): None for i in "[' ']"})
    decoded_rle = np.reshape(decode_rle(d['value']['rle']), [height, width, 4])[:, :, 3]

    img = Image.fromarray(decoded_rle)
    img.save("../static/img_annotation/" + img_name + '.png')

    # coloring the image (edit disini)
    img = Image.open(f'../static/img_annotation/' + img_name + '.png').convert('RGB')
    data = img.getdata()

    new_image = []
    for item in data:
        if item[0] in list(range(85, 256)):
            new_image.append((155,3,57))
        else:
            new_image.append(item)

    # update image data
    img.putdata(new_image)
    # img.putdata(new_image2)

    # save new image
    img.save("image_back_done.png")

# =====================================================================

# img = Image.open("../static/img_annotation/Tengkorak.png").convert("RGB")
# img2 = Image.open("../static/img_annotation/VertebraServiks.png").convert("RGB")

# d = img.getdata()
# e = img2.getdata()

# new_image = []
# color = 20
# for img_name in os.listdir(f'../static/img_annotation'):
#     # print(name)
#     img = Image.open(f'../static/img_annotation/' + img_name).convert('RGB')
#     data = img.getdata()

#     if new_image:
#         i = 0
#         for item in data:
#             # merge image
#             if item[0] in list(range(85, 256)):
#                 new_image[i] = (color,color,color)

#             i = i + 1
#         color = color + 20

#     else:
#         for item in data:
#             if item[0] in list(range(85, 256)):
#                 new_image.append((color,color,color))
#             else:
#                 new_image.append(item)

#         color = color + 20

#     print(color)

# # update image data
# img.putdata(new_image)
# # img.putdata(new_image2)

# # save new image
# img.save("image_back_done.png")


# new_image = []
# for item in d:

#     # pick colored pixels
#     if item[0] in list(range(85, 256)):
#         new_image.append((100,100,100))
#     else:
#         new_image.append(item)

# i = 0
# for item in e:

#     # merge image
#     if item[0] in list(range(85, 256)):
#         new_image[i] = (50,50,50)

#     i = i + 1

# # update image data
# img.putdata(new_image)
# # img.putdata(new_image2)

# # save new image
# img.save("image_back_done.png")
